<a href="https://colab.research.google.com/github/SirTee12/LLM-Kaggle-Google-GenAI/blob/main/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the SDK

In [13]:
!pip uninstall -qqy jupyterlab # uninstall jupyterlab in the case it conflict google colab base image
!pip install -U -q "google-genai==1.7.0" # install google genai library

import the SDK and some helpers for rendering the output. The `types` in `import types` may include custom data structures, classes or type hints psecifically to work with google AI. These custom types might represent things like model parameters, input/output formats for AI models or config settings.

In [14]:
from google import genai
from google.genai import types
from google.colab import userdata

import enum
from IPython.display import HTML, Markdown, display

## API Call Retry Implementation
Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota


1.  **Imports `retry`:** It imports the `retry` module from `google.api_core`, which provides tools for automatic retries of API calls.

2.  **Defines `is_retriable`:**
    * It creates a lambda function called `is_retriable`.
    * This function checks if an exception (`e`) is an instance of `genai.errors.APIErrpr` and if its error code (`e.code`) is either 429 (Too Many Requests) or 503 (Service Unavailable).
    * It returns `True` if both conditions are met (meaning the API call is considered retriable), and `False` otherwise.

3.  **Applies Retry Logic:**
    * It modifies the `genai.models.Models.generate_content` function by wrapping it with the `retry.retry` function.
    * The `retry.retry` function is configured with the `is_retriable` function as the `predicate`. This tells `retry.retry` to only retry the API call if the `is_retriable` function returns `True` for the encountered exception.
    * Essentially, this replaces the original `generate_content` function with a new version that automatically retries on 429 and 503 errors, making the code more robust to temporary API issues.

In [15]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in (429, 503))

genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [16]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret-label-prompt")

ModuleNotFoundError: No module named 'kaggle_secrets'

## Get the API keys and create a client

Retrieves the API key securely and create a client object that will be used to communicate with the generative AI service, using the retrieved API key for authentication.

In [18]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # retrive the APi key
client = genai.Client(api_key=GOOGLE_API_KEY)   # create a client to interact with the genai application

## Running the First Prompt

In [19]:
# generate a response
response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = "Explain AI to me like I'm a kid"
)

print(response.text)
Markdown(response.text)

Imagine you have a really, really smart puppy named Sparky. Sparky can learn new tricks, but instead of treats, he learns from lots and lots of examples! That's kind of like AI.

AI stands for Artificial Intelligence. It means making computers smart like people, but in a different way.

Here's how it works:

*   **Show Sparky lots of pictures of cats and dogs:** If you show Sparky thousands of pictures of cats and dogs and tell him which is which, he starts to learn the difference! He might notice that cats usually have pointier ears.

*   **Ask Sparky to guess:** Now you show Sparky a new picture of an animal he's never seen before. Sparky might say, "Hmm, it has pointy ears and whiskers, so I think it's a cat!" He's making a guess based on what he learned.

*   **Tell Sparky if he's right or wrong:** If Sparky guessed right, you say "Good boy!" If he guessed wrong, you tell him the correct answer. Each time he learns, he gets a little bit smarter and better at guessing.

That's what 

Imagine you have a really, really smart puppy named Sparky. Sparky can learn new tricks, but instead of treats, he learns from lots and lots of examples! That's kind of like AI.

AI stands for Artificial Intelligence. It means making computers smart like people, but in a different way.

Here's how it works:

*   **Show Sparky lots of pictures of cats and dogs:** If you show Sparky thousands of pictures of cats and dogs and tell him which is which, he starts to learn the difference! He might notice that cats usually have pointier ears.

*   **Ask Sparky to guess:** Now you show Sparky a new picture of an animal he's never seen before. Sparky might say, "Hmm, it has pointy ears and whiskers, so I think it's a cat!" He's making a guess based on what he learned.

*   **Tell Sparky if he's right or wrong:** If Sparky guessed right, you say "Good boy!" If he guessed wrong, you tell him the correct answer. Each time he learns, he gets a little bit smarter and better at guessing.

That's what AI does! It looks at tons of information, learns patterns, and then uses those patterns to make guesses or solve problems.

So, AI is like a really smart puppy that learns from lots of examples to do things like:

*   **Recognize your voice on your phone:** When you say "Hey Siri," your phone is using AI to understand what you're saying.
*   **Suggest videos for you to watch on YouTube:** YouTube's AI knows what kind of videos you like based on what you've watched before.
*   **Help doctors diagnose diseases:** AI can look at medical images and help doctors find problems that might be hard to see.
*   **Drive cars:** Some cars are being developed with AI to drive themselves!

AI isn't magic, but it's pretty cool! It's all about teaching computers to learn and make decisions like we do, but using lots and lots of information.


## Let's start a chat

In [20]:
chat = client.chats.create(model = "gemini-2.0-flash", history = [])
response = chat.send_message('Hello, my name is Ahmad')
print(response.text)

Hello Ahmad, it's nice to meet you! How can I help you today?



In [21]:
response = chat.send_message("can you tell me something interesting about dinosaurs")
print(response.text)

Okay, here's a fascinating fact about dinosaurs:

**Many dinosaurs, including the mighty Tyrannosaurus Rex, likely had feathers!**

While the image of dinosaurs is often scaly and reptilian, more and more evidence shows that feathers were much more widespread than previously thought. Scientists have found fossil evidence of feathered dinosaurs, and some even had wings! These feathers likely served different purposes:

*   **Insulation:** To keep them warm.
*   **Display:** To attract mates or intimidate rivals.
*   **Camouflage:** To blend in with their environment.

So, the next time you picture a T-Rex, imagine it with a fluffy (or even brightly colored) coat of feathers!



In [22]:
response = chat.send_message("do you still remember my name")
print(response.text)

Yes, Ahmad! I remember your name is Ahmad. Is there anything else I can help you with, Ahmad?



# Chose a model from the Gemini model family

Retrieves a list of available AI models, searches for a specific model named "gemini-2.0-flash," and if found, displays its details in a formatted JSON-like structure. The search stops once the target model is located.

In [23]:
from pprint import pprint

for model in client.models.list():
  if model.name == "models/gemini-2.0-flash":
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}


## Parameters Exploration

### Output Length

In [24]:
# set the max number of output token to 200
short_config = types.GenerateContentConfig(max_output_tokens = 200)

# create a response
response_short = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = short_config,
    contents='write a short essay on the importance of education in modern day society.'
)

print(response_short.text)

## The Cornerstone of Progress: Education in Modern Society

In the complex and rapidly evolving landscape of modern society, education stands not merely as an advantage, but as an indispensable cornerstone. Its importance extends far beyond the acquisition of knowledge and skills, serving as the engine of individual empowerment, societal progress, and global competitiveness. Investing in education is, therefore, an investment in a brighter and more prosperous future.

For the individual, education provides the tools necessary to navigate the complexities of modern life. It equips individuals with critical thinking skills, enabling them to analyze information, solve problems, and make informed decisions. Literacy and numeracy, the foundational building blocks of education, are essential for participation in the workforce, accessing essential services, and understanding the world around them. Beyond these basics, education cultivates intellectual curiosity, fosters creativity, and promo

### Temperature

How much randomness is included when selecting the next word (token) in language models depends on the "temperature" parameter.  A higher temperature causes the model to take into account a greater number of potential words, producing more inventive and diverse results.  On the other hand, a lower temperature forces the model to adhere to the most likely terms, producing language that is more concentrated and predictable.  The model turns completely deterministic when the temperature is set to 0, always choosing the word that is the most likely.  Temperature does not, however, ensure actual randomness; rather, it is a means of directing the model toward more or less random results.

In [25]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 2.0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Turquoise
 _________________________
Orange.
 _________________________
Magenta
 _________________________
Azure
 _________________________
Orange
 _________________________


The previous output shows the randomness but we would experiment with temperature value of 0. we can see from the output that it is more direct and precise and there is no randomness.

In [26]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________


### Top P

In [27]:
model_config = types.GenerateContentConfig(
    temperature = 1.0, # default config
    top_p = 0.95       # default config
)

story_prompt = "You are a senior network engineer. Explain ospf to a 5 year old"
response_topp = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents=story_prompt
)

print(response_topp.text)

Okay, imagine the internet is like a big neighborhood, and each house is a computer or device. And you want to send a letter (data) from your house to your friend's house across the neighborhood.

Now, OSPF is like a special group of mailmen. These mailmen are really smart! Instead of just guessing the best way to deliver the letter, they talk to each other.

Here's how it works:

1. **"Hello, I'm here!"**: Each mailman (router) tells all the mailmen nearby, "Hi! I'm here! These are the houses I can deliver to directly." They tell everyone their address, and how long it takes to walk to those houses. It's like saying, "I can deliver to the house next door in one minute!"

2. **Sharing the Map**: Each mailman writes down all the information they get from the other mailmen. So now they have a map of the whole neighborhood!  This map tells them which mailmen can reach which houses and how long it takes.

3. **Finding the Best Path**: When you give a letter to a mailman, they look at their

## Prompting

### Zero-shot Prompt

In [28]:
# set the prompt parameters
model_config = types.GenerateContentConfig(
  temperature = 0.1,
  top_p = 1,
  max_output_tokens = 5
)

# set the prompt
zero_short_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response_zero_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents = zero_short_prompt
)

print(response_zero_shot.text) # print the output as text

POSITIVE



#### Enum Mode
The Sentiment enum acts as a schema or a constraint. It tells the language model that the expected output should be one of these three predefined sentiment categories.

The `text/x.enum` MIME type indicates that the response should be one of the values defined in the response_schema.

In [29]:
# create a new enumeration class tha inherit from enum.Enum
class Sentiment(enum.Enum):
  POSITIVE = 'positive'
  NEUTRAL = 'neutral'
  NEGATIVE = 'negative'

response_zero_short_enum = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(

        # tell the model the ezpected output format and that the rsponse should
        # be one of thos defined in the schema
        response_mime_type = 'text/x.enum',

        # set the Sentiment as schema
        response_schema = Sentiment
    ),

    contents = zero_short_prompt
)
print(response_zero_shot.text)

POSITIVE



### Few Short Prompt

In [32]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = 'Give me a large with cheese and pineapple'

response_one_few_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        max_output_tokens = 250
    ),
    contents = [few_shot_prompt, customer_order]
)

print(response_one_few_shot.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```

